In [1]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
#现实坐标
X=np.array([[80,160],
           [160,160],
           [160,80],
           [80,80]])

In [2]:
list_x=[]
list_y=[]
def onMouse(event,x,y,flags,params):
    if event==cv2.EVENT_LBUTTONDOWN:
        print(x,y)
        list_x.append(x)
        list_y.append(y)
    return list_x,list_y

In [3]:
img = cv2.imread('D:/test_pic/hbt.jpg',1)
cv2.imshow('sample',img)
cv2.setMouseCallback('sample',onMouse)
cv2.waitKey(0)

136 100
370 103
367 334
132 332


-1

In [4]:
h,w = img.shape[:2]
print(h,w)
#定义一个变换举证
T=np.array([[1/w,0,0],[0,1/h,0],[0,0,1]])
print(T)
#获取T的逆矩阵
T_inv = np.linalg.inv(T)
print(T_inv)

1141 1187
[[8.42459983e-04 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.76424189e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[1.187e+03 0.000e+00 0.000e+00]
 [0.000e+00 1.141e+03 0.000e+00]
 [0.000e+00 0.000e+00 1.000e+00]]


In [5]:
#得到图像上的坐标了
print(list_x)
print("***********")
print(list_y)

[136, 370, 367, 132]
***********
[100, 103, 334, 332]


In [6]:
x = np.asarray([list_x])
y = np.asarray([list_y])
p1 = np.array([list_x[0],list_y[0],1])
p1_norm = T.dot(p1)
p1_norm=p1_norm[:2].tolist()
p2 = np.array([list_x[1],list_y[1],1])
p2_norm = T.dot(p2)
p2_norm=p2_norm[:2].tolist()
p3 = np.array([list_x[2],list_y[2],1])
p3_norm = T.dot(p3)
p3_norm=p3_norm[:2].tolist()
p4 = np.array([list_x[3],list_y[3],1])
p4_norm = T.dot(p4)
p4_norm=p4_norm[:2].tolist()
M = np.array([[p1_norm[0],p1_norm[1]],[p2_norm[0],p2_norm[1]],[p3_norm[0],p3_norm[1]],[p4_norm[0],p4_norm[1]]])
M

array([[0.11457456, 0.08764242],
       [0.31171019, 0.09027169],
       [0.30918281, 0.29272568],
       [0.11120472, 0.29097283]])

In [7]:
M.shape

(4, 2)

In [8]:
#构造系数矩阵
A=np.array([[M[0,0],M[0,1],1,0,0,0,-M[0,0]*8,-M[0,1]*8],
            [0,0,0,M[0,0],M[0,1],1,-M[0,0]*16,-M[0,1]*16],
            [M[1,0],M[1,1],1,0,0,0,-M[1,0]*16,-M[1,1]*16],
            [0,0,0,M[1,0],M[1,1],1,-M[1,0]*16,-M[1,1]*16],
            [M[2,0],M[2,1],1,0,0,0,-M[2,0]*16,-M[2,1]*16],
            [0,0,0,M[2,0],M[2,1],1,-M[2,0]*8,-M[2,1]*8],
            [M[3,0],M[3,1],1,0,0,0,-M[3,0]*8,-M[3,1]*8],
            [0,0,0,M[3,0],M[3,1],1,-M[3,0]*8,-M[3,1]*8]])
print(A)
#构造右侧矩阵b
b=np.array([0,0,0,0,0,0,0,0])
print(b)

[[ 0.11457456  0.08764242  1.          0.          0.          0.
  -0.91659646 -0.70113935]
 [ 0.          0.          0.          0.11457456  0.08764242  1.
  -1.83319292 -1.4022787 ]
 [ 0.31171019  0.09027169  1.          0.          0.          0.
  -4.9873631  -1.44434706]
 [ 0.          0.          0.          0.31171019  0.09027169  1.
  -4.9873631  -1.44434706]
 [ 0.30918281  0.29272568  1.          0.          0.          0.
  -4.94692502 -4.68361087]
 [ 0.          0.          0.          0.30918281  0.29272568  1.
  -2.47346251 -2.34180543]
 [ 0.11120472  0.29097283  1.          0.          0.          0.
  -0.88963774 -2.32778265]
 [ 0.          0.          0.          0.11120472  0.29097283  1.
  -0.88963774 -2.32778265]]
[0 0 0 0 0 0 0 0]


In [9]:
#求解线性方程
h = np.linalg.solve(A,b)
print(h)
h.resize(3,3)
h[2,2]=1
print(h)

[ 0.  0.  0.  0.  0.  0.  0. -0.]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0. -0.  1.]]


In [11]:
np.savetxt('result.txt',h)